In [22]:
from datasets import create_dataloader
from general import colorstr, check_img_size
import numpy as np
import os

In [2]:
import yaml

In [3]:
train_path = "../coco_kpts/train2017.txt"  # 118287 images
val_path = "../coco_kpts/val2017.txt"  # 5000 images"

# number of classes
nc = 1

names = ['person']

gs = 32 # grid size
img_size = [640, 640] # [train, test]
imgsz, imgsz_test = [check_img_size(x, gs) for x in img_size]

batch_size = 4


hyp_file = "hyp.yaml"
with open(hyp_file) as f:
    hyp = yaml.safe_load(f)  # load hyps

cache_images=False
rank=-1
world_size=1
workers=4,
image_weights=False 
quad=False
kpt_label = True

In [4]:
img_size.extend([img_size[-1]] * (2 - len(img_size)))
print(img_size)

[640, 640]


In [5]:
dataloader, dataset = create_dataloader(train_path, imgsz, batch_size, gs,
                                            hyp=hyp, augment=True, cache=cache_images, rank=rank,
                                            world_size=world_size, workers=workers,
                                            image_weights=image_weights, quad=quad, prefix=colorstr('train: '), kpt_label=kpt_label)

train: Scanning '..\coco_kpts\train2017.cache' images and labels... 56599 found, 0 missing, 0 empty, 0 corrupted: 100%|


In [6]:
mlc = np.concatenate(dataset.labels, 0)[:, 0].max()  # max label class
nb = len(dataloader)  # number of batches

In [7]:
print(mlc, nb)

0.0 14150


In [8]:
pbar = enumerate(dataloader)

In [9]:
epoch = 1
for i, (imgs, targets, paths, _) in pbar:  # batch -------------------------------------------------------------
    print(paths)
    if i>10:
        break
    ni = i + nb * epoch  # number integrated batches (since train start)

['..\\coco_kpts\\images\\train2017\\000000000036.jpg', '..\\coco_kpts\\images\\train2017\\000000000049.jpg', '..\\coco_kpts\\images\\train2017\\000000000077.jpg', '..\\coco_kpts\\images\\train2017\\000000000086.jpg']
['..\\coco_kpts\\images\\train2017\\000000000110.jpg', '..\\coco_kpts\\images\\train2017\\000000000113.jpg', '..\\coco_kpts\\images\\train2017\\000000000136.jpg', '..\\coco_kpts\\images\\train2017\\000000000151.jpg']
['..\\coco_kpts\\images\\train2017\\000000000165.jpg', '..\\coco_kpts\\images\\train2017\\000000000192.jpg', '..\\coco_kpts\\images\\train2017\\000000000241.jpg', '..\\coco_kpts\\images\\train2017\\000000000257.jpg']
['..\\coco_kpts\\images\\train2017\\000000000260.jpg', '..\\coco_kpts\\images\\train2017\\000000000294.jpg', '..\\coco_kpts\\images\\train2017\\000000000308.jpg', '..\\coco_kpts\\images\\train2017\\000000000315.jpg']
['..\\coco_kpts\\images\\train2017\\000000000322.jpg', '..\\coco_kpts\\images\\train2017\\000000000326.jpg', '..\\coco_kpts\\images\

In [24]:
import models
from config import cfg
from config import update_config
import torch
import torch.backends.cudnn as cudnn

gpu = 0
# cudnn related setting
cudnn.benchmark = cfg.CUDNN.BENCHMARK
torch.backends.cudnn.deterministic = cfg.CUDNN.DETERMINISTIC
torch.backends.cudnn.enabled = cfg.CUDNN.ENABLED

In [20]:
def update_config_from_file(cfg, args_file_path):
    cfg.defrost()
    cfg.merge_from_file(args_file_path)

    if not os.path.exists(cfg.DATASET.ROOT):
        cfg.DATASET.ROOT = os.path.join(
            cfg.DATA_DIR, cfg.DATASET.ROOT
        )

    cfg.freeze()

In [25]:
cfg_path = "experiments/coco/w32/w32_4x_reg03_bs10_512_adam_lr1e-3_coco_x140.yaml"
update_config_from_file(cfg, cfg_path)
cfg.defrost()
cfg.RANK = 0
cfg.freeze()

model = models.hrnet_dekr.get_pose_net(cfg, is_train=True)
torch.cuda.set_device(gpu)
model = model.cuda(gpu)

In [26]:
model.summary()

AttributeError: 'PoseHigherResolutionNet' object has no attribute 'summary'